<a href="https://colab.research.google.com/github/ricky-kiva/dl-deep-tf-cnn/blob/main/1_a1_cats_dogs_12500.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CNN on 12.5K Cats vs. Dogs Dataset**

Import libraries

In [1]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import matplotlib.pyplot as plt

Download dataset

In [2]:
!wget --no-check-certificate \
    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip" \
    -O "/tmp/cats-and-dogs.zip"

--2023-09-20 09:27:03--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.38.77.109, 2600:1407:3c00:398::317f, 2600:1407:3c00:397::317f
Connecting to download.microsoft.com (download.microsoft.com)|23.38.77.109|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘/tmp/cats-and-dogs.zip’

/tmp/cats-and-dogs. 100%[===================>] 786.67M  78.6MB/s    in 9.5s    

2023-09-20 09:27:12 (82.5 MB/s) - ‘/tmp/cats-and-dogs.zip’ saved [824887076/824887076]



Unzip dataset

In [3]:
local_zip = '/tmp/cats-and-dogs.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

Referencing source path

In [4]:
source_path = '/tmp/PetImages'

source_path_cats = os.path.join(source_path, 'Cat')
source_path_dogs = os.path.join(source_path, 'Dog')

Delete non-image files

In [5]:
!find /tmp/PetImages/ -type f ! -name "*.jpg" -exec rm {} +

Data exploration: number of samples

In [6]:
print(f"There are {len(os.listdir(source_path_dogs))} images of dogs.")
print(f"There are {len(os.listdir(source_path_cats))} images of cats.")

There are 12500 images of dogs.
There are 12500 images of cats.


Create training & validation directory

In [7]:
root_dir = '/tmp/cats-v-dogs'

# prevent FileExistsError in case this cell is being run several times
if os.path.exists(root_dir):
  shutil.rmtree(root_dir)

def create_train_val_dirs(root_path):
  train_dir = os.path.join(root_path, 'training')
  validation_dir = os.path.join(root_path, 'validation')

  os.makedirs(train_dir)
  os.makedirs(validation_dir)

  os.makedirs(os.path.join(train_dir, 'cats'))
  os.makedirs(os.path.join(train_dir, 'dogs'))

  os.makedirs(os.path.join(validation_dir, 'cats'))
  os.makedirs(os.path.join(validation_dir, 'dogs'))

try:
  create_train_val_dirs(root_path=root_dir)
except FileExistsError:
  print('You should not be seeing this since the upper directory is removed beforehand')

Check directory that has been made (inside *root_dir*)

In [8]:
for rootdir, dirs, files in os.walk(root_dir):
  for subdir in dirs:
    print(os.path.join(rootdir, subdir))

/tmp/cats-v-dogs/training
/tmp/cats-v-dogs/validation
/tmp/cats-v-dogs/training/dogs
/tmp/cats-v-dogs/training/cats
/tmp/cats-v-dogs/validation/dogs
/tmp/cats-v-dogs/validation/cats


Make function to split training & validation data

In [9]:
def split_data(SOURCE_DIR, TRAINING_DIR, VALIDATION_DIR, SPLIT_SIZE):
  list_dir = os.listdir(SOURCE_DIR)

  non_zero_files = []

  for fname in list_dir:
    file_path = os.path.join(SOURCE_DIR, fname)
    if os.path.isfile(file_path):
      if os.path.getsize(file_path) != 0:
        non_zero_files.append(fname)
      else:
        print(f'{fname} is zero length, so ignoring.')

  split_index = int(SPLIT_SIZE * len(non_zero_files))

  train_files, validation_files = non_zero_files[:split_index], non_zero_files[split_index:]

  for fname in train_files:
    source_path = os.path.join(SOURCE_DIR, fname)
    destination_path = os.path.join(TRAINING_DIR, fname)
    copyfile(source_path, destination_path)

  for fname in validation_files:
    source_path = os.path.join(SOURCE_DIR, fname)
    destination_path = os.path.join(VALIDATION_DIR, fname)
    copyfile(source_path, destination_path)

Split the data

In [10]:
CAT_SOURCE_DIR = "/tmp/PetImages/Cat"
DOG_SOURCE_DIR = "/tmp/PetImages/Dog"

TRAINING_DIR = "/tmp/cats-v-dogs/training"
VALIDATION_DIR = "/tmp/cats-v-dogs/validation"

TRAINING_CATS_DIR = os.path.join(TRAINING_DIR, "cats/")
TRAINING_DOGS_DIR = os.path.join(TRAINING_DIR, "dogs/")

VALIDATION_CATS_DIR = os.path.join(VALIDATION_DIR, "cats/")
VALIDATION_DOGS_DIR = os.path.join(VALIDATION_DIR, "dogs/")

# empty directories in case this cell is being run several times
if len(os.listdir(TRAINING_CATS_DIR)) > 0:
  for file in os.scandir(TRAINING_CATS_DIR):
    os.remove(file.path)
if len(os.listdir(TRAINING_DOGS_DIR)) > 0:
  for file in os.scandir(TRAINING_DOGS_DIR):
    os.remove(file.path)
if len(os.listdir(VALIDATION_CATS_DIR)) > 0:
  for file in os.scandir(VALIDATION_CATS_DIR):
    os.remove(file.path)
if len(os.listdir(VALIDATION_DOGS_DIR)) > 0:
  for file in os.scandir(VALIDATION_DOGS_DIR):
    os.remove(file.path)

split_size = .9

# run the split_data function
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, VALIDATION_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, VALIDATION_DOGS_DIR, split_size)

print(f"\nOriginal cat's directory has {len(os.listdir(CAT_SOURCE_DIR))} images")
print(f"Original dog's directory has {len(os.listdir(DOG_SOURCE_DIR))} images")

print(f"\nThere are {len(os.listdir(TRAINING_CATS_DIR))} images of cats for training")
print(f"There are {len(os.listdir(TRAINING_DOGS_DIR))} images of dogs for training")
print(f"There are {len(os.listdir(VALIDATION_CATS_DIR))} images of cats for validation")
print(f"There are {len(os.listdir(VALIDATION_DOGS_DIR))} images of dogs for validation")

666.jpg is zero length, so ignoring.
11702.jpg is zero length, so ignoring.

Original cat's directory has 12500 images
Original dog's directory has 12500 images

There are 11249 images of cats for training
There are 11249 images of dogs for training
There are 1250 images of cats for validation
There are 1250 images of dogs for validation


Make function to generate *image generators* for both training & validation data

In [11]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
  train_datagen = ImageDataGenerator(rescale=1./255.)
  validation_datagen = ImageDataGenerator(rescale=1./255.)

  train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=128,
                                                      class_mode='binary',
                                                      target_size=(150, 150))

  validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                batch_size=32,
                                                                class_mode='binary',
                                                                target_size=(150, 150))

  return train_generator, validation_generator

Call the *train_val_generators* function

In [12]:
train_generator, validation_generator = train_val_generators(TRAINING_DIR, VALIDATION_DIR)

Found 22498 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


Create the model

In [13]:
def create_model():
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
      tf.keras.layers.MaxPooling2D((2,2)),
      tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D((2,2)),
      tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D((2,2)),

      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512, activation='relu'),
      tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

  return model

Train the model

In [ ]:
model = create_model()

history = model.fit(train_generator,
                    epochs=15,
                    verbose=1,
                    validation_data=validation_generator)

Epoch 1/15
  6/176 [>.............................] - ETA: 11:57 - loss: 0.9935 - accuracy: 0.4818

Plot loss & accuracy

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, "Training Accuracy")
plt.plot(epochs, val_acc, "Validation Accuracy")
plt.title('Accuracy (Training & Validation)')
plt.legend()
plt.show()
print()

plt.plot(epochs, loss, "Training Loss")
plt.plot(epochs, val_loss, "Validation Loss")
plt.title('Loss (Training & Validation)')
plt.legend()
plt.show()

**NOTE:** Result shows it (/may) indicates overfitting

# **Download History**

To download *history.pkl* from the model to get its metadata

In [ ]:
def download_history():
  import pickle
  from google.colab import files

  with open('history.pkl', 'wb') as f:
    pickle.dump(history.history, f)

  files.download('history.pkl')

download_history()